In [78]:
import pandas as pd
import numpy as np
import json

In [110]:
with open("all.json", "r") as content:
    all = json.load(content)
u2irmap = {int(k): {int(mk): int(mv) for mk, mv in v.items()} for k, v in all.items()}

In [15]:
maxiid = 1699

In [12]:
users = sorted(list(u2irmap))

In [17]:
uirMatrix = [[0 for j in range(maxiid)] for i in users]

In [24]:
for uid, uimap in u2irmap.items():
    for iid, rating in uimap.items():
        i = users.index(uid)
        uirMatrix[i][iid-1] = rating

# Matrix Decomposition using Spark

In [76]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import DenseMatrix

In [97]:
rows = []
for uid in users:
    uimap = u2irmap[uid]
    rows.append(Vectors.sparse(maxiid+1, uimap))

In [56]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

22/12/13 22:04:01 WARN Utils: Your hostname, Kins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.86.247 instead (on interface en0)
22/12/13 22:04:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/13 22:04:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [104]:
prows = sc.parallelize(rows)

mat = RowMatrix(prows)

# Compute the top 5 singular values and corresponding singular vectors.
svd = mat.computeSVD(20, computeU=True)
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

22/12/13 22:41:36 WARN TaskSetManager: Stage 165 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:36 WARN RowMatrix: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.
22/12/13 22:41:36 WARN TaskSetManager: Stage 166 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:37 WARN TaskSetManager: Stage 167 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:37 WARN TaskSetManager: Stage 168 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:37 WARN TaskSetManager: Stage 169 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:37 WARN TaskSetManager: Stage 170 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/1

22/12/13 22:41:39 WARN TaskSetManager: Stage 225 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:39 WARN TaskSetManager: Stage 226 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:39 WARN TaskSetManager: Stage 227 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:39 WARN TaskSetManager: Stage 228 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:39 WARN TaskSetManager: Stage 229 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:39 WARN TaskSetManager: Stage 230 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:41:39 WARN TaskSetManager: Stage 231 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.

In [103]:
for U in row:
    print(U)

-1.1586602157403408e-15
2.6728584424049643e-17
1.818957954505617e-15
1.7532262425830808e-15
-1.3220814749834747e-15
1.39687615550252e-15
1.0994194137219695e-15
3.610964079374474e-16
1.538137626751889e-16
9.061534338115472e-16
-1.2840318055126045e-15
1.4109097461409308e-15
-1.6962463046954056e-15
1.794761093587517e-15
-8.493279328171862e-16
5.038561024861246e-16
-1.3691859315183224e-15
-1.771339199668182e-15
3.7800845251512284e-16
-1.2064257500633457e-15
-1.665689534144006e-05
-2.4985343011020614e-05
-8.32844767049645e-06
3.857531982647591e-05
4.215866211558974e-06
-2.569904456411406e-16
-6.163417590205359e-16
-8.145283178290217e-06
-1.3978488911408675e-06
-5.423991708600336e-06
-1.8893527616369248e-15
-1.1271691403718589e-05
-1.6907537106043295e-05
-6.007261468351392e-17
-1.1271691404402797e-05
-1.8962697391728766e-17
6.990939410323022e-06
-1.127169140317881e-05
-2.1413075815251985e-05
-1.4107164115452015e-06
-1.5566269046802293e-05
1.3500053153638987e-15
4.5848149199443204e-05
-3.1821

In [87]:
VT = DenseMatrix(V.numCols, V.numRows, V.toArray().transpose().ravel("F"))
nR = U.multiply(VT)

In [89]:
for row in nR.rows.take(3): 
    print(row.round(12)) 

22/12/13 22:27:43 WARN TaskSetManager: Stage 95 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.


[-0.          0.          0.         ...  0.00273813  0.00310905
  0.00397755]
[ 0.         -0.         -0.         ... -0.00154309  0.00013839
 -0.00163978]
[-0.          0.          0.         ...  0.00096331 -0.00030994
  0.00388266]


In [105]:
Σ = DenseMatrix(len(s), len(s), np.diag(s).ravel("F"))
V_ = DenseMatrix(V.numCols, V.numRows, V.toArray().transpose().ravel("F"))
mat_ = U.multiply(V_)

In [106]:
for row in mat_.rows.take(3): 
    print(list(map(int,row)))

22/12/13 22:41:42 WARN TaskSetManager: Stage 257 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [80]:
mat_.toArray

AttributeError: 'RowMatrix' object has no attribute 'toArray'

In [107]:
mat = RowMatrix(prows)
# Compute the top 4 principal components.
# Principal components are stored in a local dense matrix.
pc = mat.computePrincipalComponents(4)

# Project the rows to the linear space spanned by the top 4 principal components.
projected = mat.multiply(pc)

22/12/13 22:43:48 WARN TaskSetManager: Stage 258 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:43:49 WARN TaskSetManager: Stage 259 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:43:49 WARN TaskSetManager: Stage 260 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:43:49 WARN TaskSetManager: Stage 261 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
22/12/13 22:43:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/12/13 22:43:49 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [109]:
for row in mat_.rows.take(3): 
    print(list(map(int,row)))

22/12/13 22:44:14 WARN TaskSetManager: Stage 262 contains a task of very large size (1026 KiB). The maximum recommended task size is 1000 KiB.
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Matrix Decomposition

In [34]:
import numpy

def matrix_factorization(R, P, Q, K, steps=300, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T
    for step in range(steps):
        print(f"Step {step+1}/{steps}")
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [36]:
import time
start_time = time.time()

R = numpy.array(uirMatrix)
# N: num of User
N = len(R)
# M: num of Movie
M = len(R[0])
# Num of Features
K = 3

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

 
nP, nQ = matrix_factorization(R, P, Q, K)

nR = numpy.dot(nP, nQ.T)
print("--- %s seconds ---" % (time.time() - start_time))

Step 1/300
Step 2/300
Step 3/300
Step 4/300
Step 5/300
Step 6/300
Step 7/300
Step 8/300
Step 9/300
Step 10/300
Step 11/300
Step 12/300
Step 13/300
Step 14/300
Step 15/300
Step 16/300
Step 17/300
Step 18/300
Step 19/300
Step 20/300
Step 21/300
Step 22/300
Step 23/300
Step 24/300
Step 25/300
Step 26/300
Step 27/300
Step 28/300
Step 29/300
Step 30/300
Step 31/300
Step 32/300
Step 33/300
Step 34/300
Step 35/300
Step 36/300
Step 37/300
Step 38/300
Step 39/300
Step 40/300
Step 41/300
Step 42/300
Step 43/300
Step 44/300
Step 45/300
Step 46/300
Step 47/300
Step 48/300
Step 49/300
Step 50/300
Step 51/300
Step 52/300
Step 53/300
Step 54/300
Step 55/300
Step 56/300
Step 57/300
Step 58/300
Step 59/300
Step 60/300
Step 61/300
Step 62/300
Step 63/300
Step 64/300
Step 65/300
Step 66/300
Step 67/300
Step 68/300
Step 69/300
Step 70/300
Step 71/300
Step 72/300
Step 73/300
Step 74/300
Step 75/300
Step 76/300
Step 77/300
Step 78/300
Step 79/300
Step 80/300
Step 81/300
Step 82/300
Step 83/300
Step 84/300
S

# Prediction

In [39]:
test_raw = pd.read_csv('test.dat', header=None,
            names=["uid", "iid"], sep='\t', engine='python')

In [40]:
test = test_raw.values.tolist()

In [42]:
import time
start_time = time.time()
ratings = []
for index, (uid, iid) in enumerate(test):
    print(f"Predicting entry {index}/{len(test)}")
    ratings.append(nR[users.index(uid)][iid-1])
print("--- %s seconds ---" % (time.time() - start_time))

Predicting entry 0/2154
Predicting entry 1/2154
Predicting entry 2/2154
Predicting entry 3/2154
Predicting entry 4/2154
Predicting entry 5/2154
Predicting entry 6/2154
Predicting entry 7/2154
Predicting entry 8/2154
Predicting entry 9/2154
Predicting entry 10/2154
Predicting entry 11/2154
Predicting entry 12/2154
Predicting entry 13/2154
Predicting entry 14/2154
Predicting entry 15/2154
Predicting entry 16/2154
Predicting entry 17/2154
Predicting entry 18/2154
Predicting entry 19/2154
Predicting entry 20/2154
Predicting entry 21/2154
Predicting entry 22/2154
Predicting entry 23/2154
Predicting entry 24/2154
Predicting entry 25/2154
Predicting entry 26/2154
Predicting entry 27/2154
Predicting entry 28/2154
Predicting entry 29/2154
Predicting entry 30/2154
Predicting entry 31/2154
Predicting entry 32/2154
Predicting entry 33/2154
Predicting entry 34/2154
Predicting entry 35/2154
Predicting entry 36/2154
Predicting entry 37/2154
Predicting entry 38/2154
Predicting entry 39/2154
Predicting

In [49]:
ratings = list(map(round, ratings))

In [51]:
with open("test_latent.dat" ,'w') as f:
    for r in ratings:
        if r:
            f.write(f"{r}\n")
        else:
            f.write(f"{3}\n")